In [1]:
import QUANTAXIS as QA
import pandas as pd
import tushare as ts
import numpy as np
# import TA-Lib as ta
import scipy.signal as signal
import matplotlib.pyplot as plt

In [2]:
import tushare as ts
token = "488917d460bca13177c0b510492fc5fc04d0629bc76cc32a4f73fcf0"
ts.set_token(token)
pro = ts.pro_api()

In [3]:
import xlrd

In [4]:
all_stock_basic_raw = pd.read_csv(r"20220903.csv",encoding='utf-8-sig')[:-1]

In [15]:
all_stock_basic_raw.columns

Index(['代码', '名称', '涨幅%', '涨速%', '现价', '市盈(动)', '行业PE', '细分行业', '所属行业',
       'AB股总市值',
       ...
       '毛利率%', '营业利润率%', '净利润率%', '研发费用(亿)', '员工人数', '交易代码', '自选日', '自选价',
       '自选收益%', 'ab_mv_raw'],
      dtype='object', length=118)

In [5]:
all_stock_basic_raw.loc[:,'ab_mv_raw'] = all_stock_basic_raw['AB股总市值'].map(lambda x: x.strip().replace('亿',''))

In [8]:
all_stock_basic = all_stock_basic_raw[all_stock_basic_raw['ab_mv_raw'] != '--']

In [9]:
all_stock_basic.loc[:,'parent_industry'] = all_stock_basic['所属行业'].map(lambda x: x.strip().split('-')[0])

In [10]:
all_stock_basic.loc[:,'child_industry'] = all_stock_basic['所属行业'].map(lambda x: x.strip().split('-')[1])

In [11]:
all_stock_basic.loc[:,('ab_mv')] = all_stock_basic['ab_mv_raw'].map(lambda x: pd.to_numeric(x))

In [57]:
# blk_mv = pd.DataFrame(all_stock_basic.groupby(['细分行业'])['ab_mv'].sum().sort_values(ascending=False)).rename(columns={'ab_mv':'blk_mv'})

In [85]:
%time all_stock_basic.groupby(['细分行业'])['ab_mv'].agg(['sum','count'])

Wall time: 3 ms


,sum,count
细分行业,,
IT设备,4114.45,44
专用机械,17882.27,217
中成药,9537.79,74
乳制品,3098.00,16
互联网,6330.30,78
...,...,...
银行,64237.23,42
陶瓷,619.91,7
食品,14605.57,88


In [87]:
blk_mv = pd.DataFrame(all_stock_basic.groupby(['细分行业'])['ab_mv'].agg(['sum','count'])).rename(columns={'sum':'blk_mv'})

In [89]:
all_stock_basic_1 = all_stock_basic.merge(blk_mv,left_on = '细分行业',right_index=True)

In [90]:
%time all_stock_basic_1.loc[:,'mv_share'] = all_stock_basic_1['ab_mv']/all_stock_basic_1['blk_mv']*100

Wall time: 2.98 ms


In [108]:
%time top1_stocks = all_stock_basic_1.groupby(['细分行业'])\
    .apply(lambda x:x.sort_values(by='mv_share',ascending=False).head(1))\
    .filter(['代码','名称','ab_mv','blk_mv','mv_share','count']).sort_values(by='count',ascending=False)

Wall time: 288 ms


In [109]:
# 龙头A股 导出
top1_stocks.droplevel(level=1).to_csv('top1_stocks.csv',encoding='gbk')

In [100]:
%time top2_stocks = all_stock_basic_1.groupby(['细分行业'])\
    .apply(lambda x:x.sort_values(by='mv_share',ascending=False).head(2))\
    .filter(['代码','名称','ab_mv','blk_mv','mv_share','count']).sort_values(by='count',ascending=False)

Wall time: 279 ms


In [107]:
# 龙头A股 导出
top2_stocks.droplevel(level=1).to_csv('top2_stocks.csv',encoding='gbk')

In [122]:
top1_non_small_blk = top1_stocks.query('count > 4 & blk_mv > 3000')
len(top1_non_small_blk)

63

In [123]:
top1_non_small_blk

,,代码,名称,ab_mv,blk_mv,mv_share,count
细分行业,,,,,,,
软件服务,718,002230,科大讯飞,868.85,22044.60,3.941328,273
电气设备,2210,300750,宁德时代,11378.94,62863.98,18.100890,263
元器件,956,002475,立讯精密,2685.39,33136.58,8.104005,252
化工原料,2932,600309,万华化学,2730.01,29082.09,9.387255,226
专用机械,1791,300316,晶盛机电,969.49,17882.27,5.421515,217
...,...,...,...,...,...,...,...
石油加工,2707,600028,中国石化,4070.76,6130.75,66.399054,11
电信运营,3622,601728,中国电信,2942.17,5706.43,51.558856,11
空运,3509,601111,中国国航,975.29,4127.72,23.627814,9


In [118]:
top1_small_blk = top1_stocks.query('blk_mv < 3000').sort_values(by='blk_mv',ascending=False)

In [120]:
# 熊市 弱市 容易炒作小板块
len(top1_small_blk)

47

# 细分行业

In [33]:
# 超大板块 6
extra_big_blk = all_stock_basic.groupby(['细分行业'])['ab_mv'].sum().sort_values(ascending=False)[:9]
extra_big_blk

pandas.core.series.Series

In [25]:
# 大板块 12
big_blk = all_stock_basic.groupby(['细分行业'])['ab_mv'].sum().sort_values(ascending=False)[9:21]
big_blk

细分行业
化学制药    18012.01
专用机械    17882.27
汽车配件    17607.84
家用电器    16237.72
建筑工程    15022.90
保险      14884.23
通信设备    14625.45
食品      14605.57
汽车整车    14556.10
煤炭开采    14325.53
小金属     13060.15
生物制药    12629.19
Name: ab_mv, dtype: float64

In [34]:
# 中型板块 20
medium_blk = all_stock_basic.groupby(['细分行业'])['ab_mv'].sum().sort_values(ascending=False)[21:41]
medium_blk 

细分行业
石油开采    12025.27
电器仪表     9856.56
水力发电     9594.84
中成药      9537.79
航空       9057.80
新型电力     8096.65
农药化肥     7925.01
农业综合     7908.17
仓储物流     7877.44
火力发电     7413.85
全国地产     7327.68
化纤       7016.22
环境保护     6787.13
互联网      6330.30
石油加工     6130.75
电信运营     5706.43
普钢       5377.19
机械基件     5227.07
工程机械     5042.09
区域地产     4560.11
Name: ab_mv, dtype: float64

In [37]:
# 中小型板块 30
medium_samll_blk = all_stock_basic.groupby(['细分行业'])['ab_mv'].sum().sort_values(ascending=False)[41:71]
medium_samll_blk

细分行业
铝       4471.01
多元金融    4405.25
塑料      4318.46
服饰      4151.91
铜       4147.08
影视音像    4128.45
空运      4127.72
IT设备    4114.45
运输设备    4106.94
港口      4036.18
水泥      3964.67
旅游服务    3952.41
水运      3801.42
铁路      3529.90
医药商业    3488.60
家居用品    3448.22
广告包装    3386.78
供气供热    3234.91
饲料      3197.15
乳制品     3098.00
特种钢     3095.92
玻璃      3039.27
船舶      2989.42
其他建材    2957.19
矿物制品    2761.17
文教休闲    2620.09
百货      2491.49
黄金      2363.23
路桥      2246.60
机场      2207.37
Name: ab_mv, dtype: float64

In [36]:
# 小型板块
small_blk = all_stock_basic.groupby(['细分行业'])['ab_mv'].sum().sort_values(ascending=False)[71:]
small_blk

细分行业
出版业     2095.71
造纸      2083.78
日用化工    2058.14
染料涂料    1967.16
园区开发    1917.39
商贸代理    1814.38
纺织      1766.41
啤酒      1715.36
铅锌      1611.23
钢加工     1583.57
种植业     1497.44
摩托车     1432.37
综合类     1356.61
软饮料     1296.16
装修装饰    1257.84
旅游景点    1148.84
化工机械    1138.08
水务      1137.25
酒店餐饮    1100.61
其他商业    1040.46
焦炭加工    1039.38
超市连锁    1022.32
汽车服务     993.09
机床制造     914.20
橡胶       905.30
轻工机械     839.69
红黄酒      823.80
陶瓷       619.91
房产服务     615.65
批发业      608.71
纺织机械     559.14
商品城      460.67
石油贸易     368.58
农用机械     347.86
电器连锁     314.35
公共交通     262.69
渔业       241.77
林业       133.02
公路        97.49
Name: ab_mv, dtype: float64

In [42]:
categorized_blk = pd.concat([extra_big_blk,big_blk,medium_blk,medium_samll_blk,small_blk],keys=['超大板块','大板块','中型板块','中小型板块','小型板块'])

In [48]:
blk_category = categorized_blk.reset_index().drop('ab_mv',axis=1).rename(columns={'level_0':'category'})

In [50]:
blk_category.to_csv('blk_category.csv')
blk_category

,category,细分行业
0,超大板块,银行
1,超大板块,电气设备
2,超大板块,白酒
3,超大板块,元器件
4,超大板块,化工原料
...,...,...
105,小型板块,电器连锁
106,小型板块,公共交通
107,小型板块,渔业
108,小型板块,林业


In [13]:
all_stock_basic.groupby(['parent_industry'])['ab_mv'].sum().sort_values(ascending=False)

parent_industry
装备制造    120825.10
可选消费    119756.33
金融      110154.13
信息产业    107349.54
材料      102349.61
日常消费     86387.97
能源       58994.85
建筑地产     30701.57
交通运输     27924.12
社会服务     18432.89
公用事业     17128.41
商贸       11240.98
综合类       1356.61
Name: ab_mv, dtype: float64

In [15]:
# 超大板块 6个
all_stock_basic.groupby(['child_industry'])['ab_mv'].sum().sort_values(ascending=False)[:8]

child_industry
银行      64237.23
电气设备    62863.98
酿酒      46434.23
化工      44198.02
医药      40178.99
汽车类     34589.40
元器件     33136.58
半导体     27098.16
Name: ab_mv, dtype: float64

In [26]:
# 大板块 12个
big_industry = all_stock_basic.groupby(['child_industry'])['ab_mv'].sum().sort_values(ascending=False)[7:19]
big_industry

child_industry
半导体     27098.16
证券      26627.42
有色      25652.70
医疗保健    25302.00
电力      25105.34
软件服务    22044.60
工业机械    20767.04
食品饮料    18999.73
石油      18524.60
建筑      16280.74
家用电器    16237.72
煤炭      15364.91
Name: ab_mv, dtype: float64

In [25]:
# 中型板块
all_stock_basic.groupby(['child_industry'])['ab_mv'].sum().sort_values(ascending=False)[19:40]

child_industry
保险      14884.23
通信设备    14625.45
房地产     14420.83
农林牧渔    12977.55
运输服务    11556.53
建材      10581.04
钢铁      10056.68
电器仪表     9856.56
商业连锁     9426.60
航空       9057.80
交通设施     8490.15
仓储物流     7877.44
化纤       7016.22
环境保护     6787.13
互联网      6330.30
传媒娱乐     6224.16
通用机械     6141.27
纺织服饰     5918.32
电信运营     5706.43
旅游       5101.25
工程机械     5042.09
Name: ab_mv, dtype: float64

In [21]:
all_stock_basic.groupby(['child_industry'])['ab_mv'].sum().sort_values(ascending=False)[40:]

child_industry
多元金融    4405.25
IT设备    4114.45
运输设备    4106.94
家居用品    3448.22
广告包装    3386.78
供气供热    3234.91
船舶      2989.42
矿物制品    2761.17
文教休闲    2620.09
造纸      2083.78
日用化工    2058.14
商贸代理    1814.38
综合类     1356.61
水务      1137.25
酒店餐饮    1100.61
公共交通     262.69
Name: ab_mv, dtype: float64

In [17]:
all_stock_basic.groupby(['细分行业'])['ab_mv'].sum().sort_values(ascending=False)[7:20]

细分行业
医疗保健    25302.00
软件服务    22044.60
化学制药    18012.01
专用机械    17882.27
汽车配件    17607.84
家用电器    16237.72
建筑工程    15022.90
保险      14884.23
通信设备    14625.45
食品      14605.57
汽车整车    14556.10
煤炭开采    14325.53
小金属     13060.15
Name: ab_mv, dtype: float64

In [14]:
#超大板块
电气设备    装备制造 查看相关性高的板块
酿酒      46434.23
化工      44198.02
医药      40178.99
汽车类     34589.40
元器件     33136.58
supper_big_industry_code = ['880446','880380','880335','880400','880390','880492']

In [6]:
pd.read_csv('Industry Classification.csv')

,代码,名称,大板块
0,880446,电气设备,制造
1,880471,银行,金融
2,880381,白酒,食品消费
3,880492,元器件,电子
4,880336,化工原料,化工
...,...,...,...
105,880409,电器连锁,商业
106,880362,渔业,第一产业
107,880453,公共交通,公共事业
108,880363,林业,第一产业


In [7]:
# 获得所有A股的市值
df_total_mv = pro.query('daily_basic', ts_code='', trade_date='20220830',fields='ts_code,total_mv,circ_mv,dv_ratio,volume_ratio,pe,pb')

In [8]:
import re
# regex = re.compile('\d*.\w*')

In [9]:
# %time df_total_mv.loc[:,'code3'] = df_total_mv['ts_code'].map(lambda x: x.strip('.,SZ,SH,BJ'))
# Wall time: 6 ms

In [10]:
# %time df_total_mv.loc[:,'code2'] = df_total_mv['ts_code'].map(lambda x: re.sub("(\.\w{2})","",x))
# Wall time: 10 ms

In [11]:
%time df_total_mv.loc[:,'code'] = df_total_mv['ts_code'].map(lambda x: x.replace('.SH','').replace('.SZ','').replace('.BJ',''))

Wall time: 3 ms


In [12]:
df_total_mv

,ts_code,volume_ratio,pe,pb,dv_ratio,total_mv,circ_mv,code
0,603360.SH,0.66,17.2847,3.3776,3.4678,4.329677e+05,4.278404e+05,603360
1,002928.SZ,0.49,NaN,4.1853,0.5545,1.114924e+06,1.114924e+06,002928
2,000806.SZ,0.53,NaN,NaN,0.0000,2.529797e+05,1.608401e+05,000806
3,000875.SZ,1.03,42.8711,1.7147,0.0000,1.930824e+06,1.692408e+06,000875
4,600521.SH,0.60,60.3693,4.7565,0.9884,2.943214e+06,2.885952e+06,600521
...,...,...,...,...,...,...,...,...
4880,600581.SH,0.66,5.7879,1.6530,0.0000,6.836671e+05,6.744751e+05,600581
4881,000721.SZ,1.20,NaN,3.9619,0.0000,2.622788e+05,1.992304e+05,000721
4882,002165.SZ,3.61,121.1569,1.7090,1.6327,3.602822e+05,3.563777e+05,002165
4883,300842.SZ,0.60,73.4758,7.1722,0.0000,6.902000e+05,4.797661e+05,300842


In [13]:
%time all_blk_component = QA.QA_fetch_stock_block_adv().view_block

Wall time: 1.32 s


In [14]:
index_list = QA.QA_fetch_index_list()

In [15]:
# index_list.loc['880610']
index_list.shape

(1220, 7)

In [16]:
all_blk_component.shape

(1212,)

In [17]:
index_list_df = index_list.reset_index(drop=True).set_index('name').filter(['code'])

In [18]:
%time index_list_df.merge(all_blk_component.rename('component').to_frame(),left_index=True,right_index=True)

Wall time: 8.99 ms


,code,component
3D打印,880569,"[000821, 000837, 000928, 000938, 000969, 00098..."
5G概念,880506,"[000004, 000049, 000063, 000066, 000070, 00015..."
BIPV概念,880977,"[000012, 000027, 000040, 000055, 000058, 00059..."
C2M概念,880717,"[000726, 002094, 002154, 002397, 002489, 00250..."
CXO概念,880729,"[000813, 002370, 002821, 300012, 300149, 30034..."
...,...,...
高速公路,880662,"[000429, 000548, 000755, 000828, 000900, 00196..."
鸡肉,880764,"[000048, 000529, 000735, 000876, 002100, 00223..."
鸿蒙概念,880722,"[000004, 000158, 000676, 000997, 002123, 00221..."
黄金,880328,"[000506, 000975, 002155, 002237, 600311, 60048..."


In [19]:
%time index_name_code_component = index_list_df.merge(all_blk_component.rename('component').to_frame(),left_index=True,right_index=True)

Wall time: 3.97 ms


In [20]:
index_name_code_component.sort_values('code')

,code,component
上证380,000009,"[600007, 600008, 600015, 600018, 600021, 60002..."
上证180,000010,"[600000, 600009, 600010, 600011, 600016, 60001..."
上证50,000016,"[600010, 600028, 600030, 600031, 600036, 60004..."
上证中盘,000044,"[600000, 600009, 600011, 600016, 600019, 60002..."
高端装备,000097,"[000039, 000157, 000338, 000400, 000410, 00041..."
...,...,...
医美概念,880973,"[000150, 000516, 000538, 000615, 000650, 00071..."
烟草概念,880974,"[000812, 000910, 002012, 002161, 002191, 00231..."
有机硅,880975,"[000830, 001212, 002054, 002091, 002132, 00221..."
新冠检测,880976,"[000513, 000710, 002022, 002030, 002038, 00204..."


In [21]:
# 有重复
index_name_code_component.loc['煤炭开采']

,code,component
煤炭开采,880302,"[000552, 000571, 000937, 000983, 002128, 60012..."
煤炭开采,880610,"[000552, 000571, 000937, 000983, 002128, 60012..."


In [24]:
industry_list_df= index_name_code_component.set_index('code').sort_index().loc['880302':'880497']

In [25]:
# 第一个
industry_list_df.loc['880302']

component    [000552, 000571, 000937, 000983, 002128, 60012...
Name: 880302, dtype: object

In [26]:
# 最后一个
industry_list_df.loc['880497']

component    [000034, 000038, 000632, 000839, 000881, 00090...
Name: 880497, dtype: object

In [27]:
industry_list_df

,component
code,
880302,"[000552, 000571, 000937, 000983, 002128, 60012..."
880303,"[000723, 600408, 600725, 600740, 600792, 60101..."
880306,"[000601, 000791, 000993, 002039, 600025, 60010..."
880307,"[000027, 000037, 000531, 000539, 000543, 00060..."
880308,"[000040, 000155, 000537, 000591, 000862, 00125..."
...,...
880491,"[001270, 001309, 002077, 002079, 002119, 00215..."
880492,"[000020, 000032, 000045, 000050, 000100, 00041..."
880493,"[000004, 000158, 000409, 000503, 000555, 00063..."


In [28]:
industry_list_df.iloc[0]

component    [000552, 000571, 000937, 000983, 002128, 60012...
Name: 880302, dtype: object

In [29]:
type(industry_list_df.iloc[0])

pandas.core.series.Series

In [30]:
idx_code = industry_list_df.index.values[0]

In [31]:
# it's series
component = industry_list_df.iloc[0]

In [32]:
component_list = component.iloc[0]

In [33]:
type(industry_list_df)

pandas.core.frame.DataFrame

In [34]:
list([idx_code]*10)

['880302',
 '880302',
 '880302',
 '880302',
 '880302',
 '880302',
 '880302',
 '880302',
 '880302',
 '880302']

In [148]:
all_component = []

In [150]:
all_component

[]

In [118]:
%time stock_index_mapping = pd.DataFrame(columns=['index_code'])
all_component = []
for index, row in industry_list_df.iterrows():
    component = row.iloc[0]
    all_component.extend(component)
    one_blk = pd.DataFrame(list([pd.to_numeric(index)]*len(component)),index=component,columns=['index_code'])
#     stock_index_mapping = pd.concat([stock_index_mapping,one_blk],axis=1)
    stock_index_mapping = stock_index_mapping.append(one_blk)    

Wall time: 994 µs


In [109]:
# stock_index_mapping.drop(['duplicate'],axis=1,inplace=True)

In [129]:
type(stock_index_mapping.loc['000552'].values[0])

int

In [132]:
type(stock_index_mapping['index_code'][0])

int

In [139]:
test = stock_index_mapping.reset_index().groupby(['index']).rank(ascending=1)

DataError: No numeric types to aggregate

In [137]:
test[test['index_code']==2]

,index_code
index,
000068,2
000638,2
000882,2
000883,2
000920,2
002122,2
002182,2
002357,2
002575,2


In [65]:
df_total_mv_code = df_total_mv.set_index('code')

In [92]:
df_total_mv_code.loc[:,'duplicate'] = df_total_mv_code.index.duplicated()

In [94]:
df_total_mv_code[df_total_mv_code['duplicate'] == True]

,ts_code,volume_ratio,pe,pb,dv_ratio,total_mv,circ_mv,duplicate
code,,,,,,,,


In [95]:
stock_index_mapping.loc[:,'duplicate'] = stock_index_mapping.index.duplicated()

In [96]:
stock_index_mapping[stock_index_mapping['duplicate'] == True]

,index_code,duplicate
600997,880303,True
000883,880307,True
600256,880313,True
002182,880329,True
002918,880348,True
300106,880373,True
605388,880374,True
300915,880375,True
605300,880375,True
002718,880399,True


In [105]:
stock_index_mapping.loc['600997']

,index_code,duplicate
600997,880302,False
600997,880303,True


In [89]:
stock_mv_index = pd.concat([df_total_mv_code,stock_index_mapping],axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [44]:
stock_mv_index[~stock_mv_index['index_code'].isnull()]

,ts_code,volume_ratio,pe,pb,dv_ratio,total_mv,circ_mv,index_code
000552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,880302
000571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,880302
000937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,880302
000983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,880302
002128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,880302
...,...,...,...,...,...,...,...,...
002575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,880497
300208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,880497
600226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,880497
600676,NaN,NaN,NaN,NaN,NaN,NaN,NaN,880497


In [82]:
df4 = pd.DataFrame([['bird', 'polly'], ['monkey', 'george']],
                   columns=['animal', 'name'],index=['a','b'])

In [83]:
df4

,animal,name
a,bird,polly
b,monkey,george


In [84]:
df1 = pd.DataFrame([['a', 1], ['b', 2]],
                   columns=['letter', 'number'],index=['a','b'])
df1

,letter,number
a,a,1
b,b,2


In [85]:
pd.concat([df1,df4],axis=1)

,letter,number,animal,name
a,a,1,bird,polly
b,b,2,monkey,george


array([list(['000552', '000571', '000937', '000983', '002128', '600121', '600123', '600188', '600348', '600395', '600397', '600403', '600508', '600546', '600758', '600971', '600985', '600997', '601001', '601088', '601101', '601225', '601666', '601699', '601898', '601918'])],
      dtype=object)

In [85]:
component_list.values.

(1,)

In [ ]:
QA.QA_fetch_c

In [75]:
list(idx_code*len(component_list))

['8', '8', '0', '3', '0', '2']

In [73]:
pd.DataFrame(idx_code*len(component_list),index=component_list)

ValueError: DataFrame constructor not properly called!

component    [000034, 000038, 000632, 000839, 000881, 00090...
Name: 880497, dtype: object

In [4]:
industry_classification = pd.read_csv('Industry Classification.csv',dtype={'代码':str})

In [5]:
target_index = industry_classification[industry_classification['大板块']=='制造']['代码'].to_list()

In [6]:
target_index

['880446',
 '880445',
 '880392',
 '880448',
 '880439',
 '880447',
 '880350',
 '880368',
 '880441',
 '880438',
 '880442',
 '880443',
 '880444']

In [7]:
test = QA.QA_fetch_index_day_adv(target_index,'2022-01-1','2022-08-29')

In [8]:
df_test = test.reset_index()

In [9]:
new_df = df_test.pivot(index='date',columns='code',values='close')

In [10]:
new_df.corr()

code,880350,880368,880392,880438,880439,880441,880442,880443,880444,880445,880446,880447,880448
code,,,,,,,,,,,,,
880350,1.000000,0.796932,0.447125,0.690140,0.438129,0.318430,0.678466,0.809292,0.712464,0.529191,0.404032,0.892746,0.943824
880368,0.796932,1.000000,0.797623,0.853455,0.766074,0.663944,0.828991,0.911112,0.877429,0.819748,0.738172,0.889190,0.794962
880392,0.447125,0.797623,1.000000,0.871407,0.957337,0.861877,0.793996,0.778685,0.723601,0.953203,0.939227,0.667754,0.489103
880438,0.690140,0.853455,0.871407,1.000000,0.911816,0.815590,0.907279,0.840414,0.795601,0.940451,0.807588,0.758060,0.713147
880439,0.438129,0.766074,0.957337,0.911816,1.000000,0.908732,0.771453,0.731663,0.751855,0.981075,0.948863,0.597998,0.448946
880441,0.318430,0.663944,0.861877,0.815590,0.908732,1.000000,0.722994,0.631607,0.681371,0.910931,0.861845,0.451227,0.320251
880442,0.678466,0.828991,0.793996,0.907279,0.771453,0.722994,1.000000,0.814490,0.680973,0.842993,0.679729,0.792074,0.756422
880443,0.809292,0.911112,0.778685,0.840414,0.731663,0.631607,0.814490,1.000000,0.854752,0.785756,0.720715,0.862914,0.812719
880444,0.712464,0.877429,0.723601,0.795601,0.751855,0.681371,0.680973,0.854752,1.000000,0.781236,0.731461,0.746637,0.661675


In [30]:
idx_list = QA.QA_fetch_index_list_adv()


,code,volunit,decimal_point,name,pre_close,sse,sec
code,,,,,,,
000001,000001,100.0,2.0,上证指数,2822.442383,sh,index_cn
000002,000002,100.0,2.0,Ａ股指数,2958.152588,sh,index_cn
000003,000003,100.0,2.0,Ｂ股指数,210.989807,sh,index_cn
000004,000004,100.0,2.0,工业指数,2291.368164,sh,index_cn
000005,000005,100.0,2.0,商业指数,2728.330322,sh,index_cn
...,...,...,...,...,...,...,...
000109,000109,100.0,2.0,380医药,12490.842773,sh,index_cn
000110,000110,100.0,2.0,380金融,3080.125732,sh,index_cn
000111,000111,100.0,2.0,380信息,7211.254883,sh,index_cn


In [31]:
idx_list['code'].to_list()

['000001',
 '000002',
 '000003',
 '000004',
 '000005',
 '000006',
 '000007',
 '000008',
 '000009',
 '000010',
 '000011',
 '000012',
 '000013',
 '000015',
 '000016',
 '000017',
 '000018',
 '000019',
 '000020',
 '000021',
 '000022',
 '000025',
 '000026',
 '000027',
 '000028',
 '000029',
 '000030',
 '000031',
 '000032',
 '000033',
 '000034',
 '000035',
 '000036',
 '000037',
 '000038',
 '000039',
 '000040',
 '000041',
 '000042',
 '000043',
 '000044',
 '000045',
 '000046',
 '000047',
 '000048',
 '000049',
 '000050',
 '000051',
 '000052',
 '000053',
 '000054',
 '000055',
 '000056',
 '000057',
 '000058',
 '000059',
 '000060',
 '000061',
 '000062',
 '000063',
 '000064',
 '000065',
 '000066',
 '000067',
 '000068',
 '000069',
 '000070',
 '000071',
 '000072',
 '000073',
 '000074',
 '000075',
 '000076',
 '000077',
 '000078',
 '000079',
 '000090',
 '000091',
 '000092',
 '000093',
 '000094',
 '000095',
 '000096',
 '000097',
 '000098',
 '000099',
 '000100',
 '000101',
 '000102',
 '000103',
 '000104',